<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/AAI_MULTIAGENT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install 'crewai[tools]' -q
!pip install crewai -q
!pip install 'serpapi' -q
!pip install 'PyPDF2' -q
!pip install colab-env -q

In [4]:
import os
from crewai import Agent, Task, Crew, Process
from crewai_tools import PDFSearchTool
from typing import List
import colab_env
import warnings
warnings.filterwarnings('ignore')

import logging
logging.getLogger('chromadb').setLevel(logging.ERROR)

# Ensure you have your OPENAI_API_KEY set as an environment variable
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# Use a publicly available PDF URL for demonstration
sample_pdf_url = "https://www.orimi.com/pdf-test.pdf"
#pdf_search_tool = PDFSearchTool(pdf=sample_pdf_url)


pdf_search_tool = PDFSearchTool(pdf=sample_pdf_url, search_kwargs={"k": 1})

# --- Define all possible agents ---
# Research Agents
senior_researcher = Agent(
    role='Senior Researcher',
    goal='Analyze key technological trends from a provided, internal dataset.',
    backstory='A meticulous researcher who specializes in analyzing internal data sources.',
    verbose=False ,
    allow_delegation=False,
    # This agent no longer uses an external search tool
    tools=[]
)

summarizer_agent = Agent(
    role='Document Summarizer',
    goal='Summarize key information from provided PDF documents.',
    backstory='An expert in quickly digesting long documents and extracting the most important points.',
    verbose=False ,
    allow_delegation=False,
    tools=[pdf_search_tool]
)

data_analyst = Agent(
    role='Data Analyst',
    goal='Extract key insights and statistical data from research findings.',
    backstory='An expert in quantitative analysis, finding patterns and metrics in complex datasets.',
    verbose=False ,
    allow_delegation=False
)

# Creative Agents
creative_writer = Agent(
    role='Creative Content Writer',
    goal='Draft engaging and persuasive content based on provided information.',
    backstory='An imaginative writer who can turn dry facts into compelling stories.',
    verbose=False ,
    allow_delegation=False
)

social_media_strategist = Agent(
    role='Social Media Strategist',
    goal='Develop a social media plan to promote the generated content.',
    backstory='An expert in digital marketing, crafting effective strategies for online platforms.',
    verbose=False ,
    allow_delegation=False
)

# --- The Classifier and Orchestration Logic ---
def get_crew(user_input: str) -> Crew:
    """Classifies the user input and returns the appropriate Crew."""

    if "research" in user_input.lower() or "analyze" in user_input.lower() or "document" in user_input.lower():
        print("### Classifying as a RESEARCH & DOCUMENT ANALYSIS task. Assembling Research Crew. ###")

        # Define tasks for the research team
        research_task = Task(
            description=f"Analyze the following pre-defined dataset and identify key trends: 'In 2023, the generative AI market grew by 35% driven by a 20% increase in enterprise adoption. Key players include OpenAI, Google, and Anthropic. A major challenge remains model explainability.'",
            agent=senior_researcher,
            expected_output='A summary of key trends and growth metrics from the provided dataset.'
        )

        summarize_task = Task(
            description=f"Using the PDF document available at {sample_pdf_url}, summarize its main points and purpose.",
            agent=summarizer_agent,
            expected_output='A concise summary of the PDF document.'
        )

        analysis_task = Task(
            description="Analyze both the internal research findings and the PDF summary to provide a combined report. Highlight key insights from both sources.",
            agent=data_analyst,
            expected_output='A detailed report combining insights from the internal research and the PDF summary.'
        )

        research_crew = Crew(
            agents=[senior_researcher, summarizer_agent, data_analyst],
            tasks=[research_task, summarize_task, analysis_task],
            process=Process.sequential,
            verbose=False
        )
        return research_crew

    elif "write" in user_input.lower() or "content" in user_input.lower() or "blog" in user_input.lower():
        print("### Classifying as a CONTENT CREATION task. Assembling Creative Crew. ###")

        # Define tasks for the creative team
        writing_task = Task(
            description=f"Write a compelling blog post based on the user's request: '{user_input}'. The post should be engaging and informative.",
            agent=creative_writer,
            expected_output='A well-structured blog post of at least 500 words, formatted in markdown.'
        )

        social_media_task = Task(
            description="Based on the blog post, create a social media strategy. Provide three different post variations for Twitter, LinkedIn, and Facebook to promote the content.",
            agent=social_media_strategist,
            expected_output='A social media strategy with posts for different platforms.'
        )

        creative_crew = Crew(
            agents=[creative_writer, social_media_strategist],
            tasks=[writing_task, social_media_task],
            process=Process.sequential,
            verbose=False
        )
        return creative_crew

    else:
        # Default fallback
        print("### Classification failed. Using a general-purpose agent. ###")

        general_agent = Agent(
            role='General AI Assistant',
            goal='Provide a helpful and concise response to the user query.',
            backstory='A versatile assistant capable of handling a wide range of topics.',
            verbose=False ,
            allow_delegation=False
        )

        general_task = Task(
            description=f"Respond to the user's query: '{user_input}'.",
            agent=general_agent,
            expected_output='A helpful and concise response to the user query.'
        )

        general_crew = Crew(
            agents=[general_agent],
            tasks=[general_task],
            process=Process.sequential,
            verbose=False
        )
        return general_crew

# --- Main execution block ---
if __name__ == "__main__":
    # This input will trigger the new research and document analysis crew
    user_request = "Can you analyze recent market data for AI and also summarize the key points of the provided PDF document?"

    # The classification step will now dynamically create the research crew.
    selected_crew = get_crew(user_request)

    print("\n\n### Running the selected crew ###")
    result = selected_crew.kickoff()

    print("\n\n################################################")
    print("## Final Result from the Orchestration:")
    print("################################################\n")
    print(result)

### Classifying as a RESEARCH & DOCUMENT ANALYSIS task. Assembling Research Crew. ###


### Running the selected crew ###


################################################
## Final Result from the Orchestration:
################################################

**Combined Report on Generative AI Market Insights**

**Overview:**
In 2023, the generative AI market has shown remarkable growth and significant trends that warrant a focused examination. This report synthesizes findings from internal research and available data, outlining key insights and patterns essential for understanding the current landscape of generative AI technologies.

**Market Growth:**
The generative AI market has experienced an impressive growth rate of **35%** in 2023. This growth trajectory indicates a robust and increasing adoption of generative AI solutions across various sectors. The rise demonstrates a clear technological shift as organizations seek innovative avenues to leverage AI capabilities for improved